Exploring the oldest businesses in the world via Pandas

Datasets are collected by BusinessFinancing.co.uk.



# Datasets
<p>Our datasets, which are all located in the <code>datasets</code> directory, contain the following information: </p>
<h3 id="businessesandnew_businesses"><code>businesses</code> and <code>new_businesses</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>business</code></td>
<td>varchar</td>
<td>Name of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>year_founded</code></td>
<td>int</td>
<td>Year the business was founded.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>char</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
</tbody>
</table>

<h3 id="countries"><code>countries</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>country_code</code></td>
<td>varchar</td>
<td>ISO 3166-1 3-letter country code.</td>
</tr>
<tr>
<td style="text-align:left;"><code>country</code></td>
<td>varchar</td>
<td>Name of the country.</td>
</tr>
<tr>
<td style="text-align:left;"><code>continent</code></td>
<td>varchar</td>
<td>Name of the continent that the country exists in.</td>
</tr>
</tbody>
</table>
<h3 id="categories"><code>categories</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>category_code</code></td>
<td>varchar</td>
<td>Code for the category of the business.</td>
</tr>
<tr>
<td style="text-align:left;"><code>category</code></td>
<td>varchar</td>
<td>Description of the business category.</td>
</tr>
</tbody>
</table>

# Top five of the oldest in the world

In [2]:
import pandas as pd

businesses = pd.read_csv('datasets/businesses.csv')

# Sort businesses from oldest businesses to youngest
sorted_businesses = businesses.sort_values('year_founded')

sorted_businesses.head()

,business,year_founded,category_code,country_code
64,Kongō Gumi,578,CAT6,JPN
94,St. Peter Stifts Kulinarium,803,CAT4,AUT
107,Staffelter Hof Winery,862,CAT9,DEU
106,Monnaie de Paris,864,CAT12,FRA
103,The Royal Mint,886,CAT12,GBR


In [4]:
# Let's add country names and continents
countries = pd.read_csv('datasets/countries.csv')
businesses_countries = sorted_businesses.merge(countries, on='country_code')

businesses_countries.head()

,business,year_founded,category_code,country_code,country,continent
0,Kongō Gumi,578,CAT6,JPN,Japan,Asia
1,St. Peter Stifts Kulinarium,803,CAT4,AUT,Austria,Europe
2,Staffelter Hof Winery,862,CAT9,DEU,Germany,Europe
3,Monnaie de Paris,864,CAT12,FRA,France,Europe
4,The Royal Mint,886,CAT12,GBR,United Kingdom,Europe


# Top five of the oldest in the North America

In [5]:
# Filter businesses_countries to include countries in North America only
north_america = businesses_countries[businesses_countries['continent'] == 'North America']
north_america.head()

,business,year_founded,category_code,country_code,country,continent
22,La Casa de Moneda de México,1534,CAT12,MEX,Mexico,North America
28,Shirley Plantation,1638,CAT1,USA,United States,North America
33,Hudson's Bay Company,1670,CAT17,CAN,Canada,North America
35,Mount Gay Rum,1703,CAT9,BRB,Barbados,North America
40,Rose Hall,1770,CAT19,JAM,Jamaica,North America


# The oldest business on each continent

In [33]:
# Find
continent = businesses_countries.groupby('continent')[['year_founded']].agg(min)

# Merge
merged_continent = continent.merge(businesses_countries)

# Remove technical columns
subset_merged_continent = merged_continent[['continent', 'country', 'business', 'year_founded']]
subset_merged_continent

,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,Oceania,Australia,Australia Post,1809
5,South America,Peru,Casa Nacional de Moneda,1565


# Exploring missing data

The dataset "businesses" contain less countries, than the dataset "countries":

In [21]:
businesses['country_code'].nunique()

163

In [22]:
countries['country_code'].nunique()

195

Let's find countries that are missing from businesses dataset:

In [23]:
# Outer merge
all_countries = businesses.merge(countries, on='country_code', how='outer')

# Filter to include only countries without oldest businesses
missing_countries = all_countries[all_countries['business'].isnull()]

# Display
missing_countries_series = pd.Series(missing_countries['country'])
missing_countries_series

163                              Angola
164                 Antigua and Barbuda
165                             Bahamas
166                  Dominican Republic
167                             Ecuador
168                                Fiji
169     Micronesia, Federated States of
170                               Ghana
171                              Gambia
172                             Grenada
173           Iran, Islamic Republic of
174                          Kyrgyzstan
175                            Kiribati
176               Saint Kitts and Nevis
177                              Monaco
178                Moldova, Republic of
179                            Maldives
180                    Marshall Islands
181                               Nauru
182                               Palau
183                    Papua New Guinea
184                            Paraguay
185                 Palestine, State of
186                     Solomon Islands
187                            Suriname


# Adding missing data

We have found info on a few of the countries with missing data, it is in "new_businesses.csv":

In [27]:
new_businesses = pd.read_csv('datasets/new_businesses.csv')
new_businesses

,business,year_founded,category_code,country_code
0,Fiji Times,1869,CAT13,FJI
1,J. Armando Bermúdez & Co.,1852,CAT9,DOM


In [26]:
# Add the data in new_businesses to the existing businesses
all_businesses = pd.concat([businesses, new_businesses])

# Check if we have 163+2=165 unique countries now
all_businesses['country_code'].nunique()

165

In [31]:
# Merge and filter to find countries with missing business data
new_all_countries = all_businesses.merge(countries, on='country_code', how='outer')
new_missing_countries = new_all_countries[new_all_countries['business'].isnull()]

# Group by continent and create a "count_missing" column
count_missing = new_missing_countries.groupby('continent').agg({'country_code': 'nunique'})
count_missing.columns = ["count_missing"]
count_missing

,count_missing
continent,
Africa,3
Asia,7
Europe,2
North America,5
Oceania,10
South America,3


# Exploring categories of businesses


In [49]:
# Import names of categories
categories = pd.read_csv("datasets/categories.csv")
businesses_categories = businesses.merge(categories, on='category_code')

# Find the number of oldest businesses in each category
count_business_cats = businesses_categories.groupby('category').agg({'business': 'nunique'})
count_business_cats.columns = ["count"]

# Find the cumulative years businesses from each category have been operating
years_business_cats = businesses_categories.groupby("category").agg({'year_founded':'sum'})
years_business_cats.columns = ["total_years_in_business"]
from datetime import datetime
currentYear = datetime.now().year
years_business_cats["total_years_in_business"] = currentYear * count_business_cats["count"] - years_business_cats["total_years_in_business"]

# Display
business_cats = count_business_cats.join(years_business_cats)
business_cats

,count,total_years_in_business
category,,
Agriculture,6,1469
Aviation & Transport,19,1839
Banking & Finance,37,4549
"Cafés, Restaurants & Bars",6,3606
Conglomerate,3,398
Construction,2,1623
Consumer Goods,3,826
Defense,3,572
"Distillers, Vintners, & Breweries",22,7214


# Old Cafés, Restaurants & Bars

